## GPT

GPT 的论文名为《Improving Language Understanding by Generative Pre-Training》。其核心思想是先通过**无标签的文本去训练生成语言模型**，再根据具体的 NLP 任务（如文本蕴含、QA、文本分类等），使用**有标签的数据对模型进行 fine-tuning**。

### 论文原理

具体来说，论文提出了半监督的方法，即结合了无监督的预训练和有监督的 fine-tuning。论文采用两阶段训练，首先，在未标记数据集上训练语言模型来学习神经网络模型的初始参数，随后使用相应 NLP 任务中的有标签数据将这些参数微调，来适应当前的任务。

### 模型结构

模型使用了多层 *Transformer decoder*，下图是 GPT 语言模型的结构：
<img src="/images/gpt.png" width="550">
训练的两个阶段如下：

1. 无监督的预训练

对于无标签的文本 $\mathcal{U}=\{u_1,\cdots, u_n\}$，最大化语言模型的极大似然函数：
$$L_1(\mathcal{U})=\sum_{i}\log P(u_i|u_{i-k},\cdots,u_{i-1};\Theta)$$
这里的 k 是文本上下文窗口的大小，条件概率 $P$ 用参数为 $\Theta$ 的神经网络建模，这些参数使用 SGD 算法优化。

论文中使用的是多层 Transformer 的 decoder 的语言模型，input 为词嵌入以及单词 token 的位置信息，再对 Transformer block 的输出向量做 softmax，output 为词的概念分布。具体公式如下：
<img src="/images/gpt_3.1.png" width="320">
其中，$U=(u_{-k},\cdots,u_{-1})$ 是 tokens 的上下文向量，$n$ 是层数，$W_e$ 是 token embedding 矩阵，其实 $UW_e$ 就是求 tokens 的 embedding，$W_p$ 是位置 embedding 矩阵。

2. 有监督的 fine-tuning

在对模型预训练之后，采用有监督的目标任务对模型参数 fine-tuning。假设有带标签的数据集 $\mathcal{C}$，每条数据为一个单词序列 $x^1,\cdots,x^m$，以及相应的标签 $y$。通过之前预训练的模型获取输出向量 $h_l^m$，再送入线性输出层（参数为 $W_y$）来预测 $y$：
$$P(y|x^1,\cdots,x^m)=softmax(h_l^mW_y)$$
最大化以下函数：
$$L_2(\mathcal{C})=\sum_{(x,y)}\log P(y|x^1,\cdots,x^m)$$
文章说，用语言模型来辅助 fine-tuning 过程有两个好处：（1）可以提高监督模型的泛性，（2）可以加速收敛。

最后，将两阶段的目标函数通过超参 $\lambda$ 相加训练整个模型：
$$L_3(\mathcal{C})=L_2(\mathcal{C})+\lambda * L_1(\mathcal{C})$$

3. 具体任务的模型微调

对于文本分类任务，只需要在预训练模型上微调。对于 QA 任务或文本蕴含，因为预训练模型是在连续序列上训练，需要做一些调整，修改输入结构，将输入转化为有序序列输入。

1）文本蕴含：将前提 $p$ 和假设 $h$ 序列拼接，中间用 ($) 符号来分隔两个序列；

2）文本相似度：分别将两个序列输入，通过模型输出两个序列的特征向量，再逐元素相加输入线性层；

3）QA 和常识推理：给定上下文文本 $z$，问题 $q$，一组可能的候选答案 $\{a_k\}$，将上下文文本、问题以及每个候选答案拼接起来，得到这样一个序列 $[z;q;\$;a_k]$，再将该序列输入预训练模型，经 softmax 层得到候选答案的分布。

### Experiments

1. 无监督预训练

使用 BooksCorpus 数据集来训练语言模型，该数据集包含 7000 多本各种文体的书，更重要的是它包含长段连续文本，这就使得生成模型可以学习长段信息。文章指出，ELMo 模型使用了 1B Word Benchmark，两者之间大小差不多，但后者在句子层 shuffle 了，这就破坏了长段结构。

预训练语言模型的 perplexity 是 18.4，非常好的一个结果。

2. Model specifications

模型和原始 Transformer 类似，使用 masked 自注意力头（768 维的 states，12 个注意力头）来训练 12 层的只有 decoder 的 transformer。对于 position-wise feed-forward，使用 3072 维的 inner states。使用 Adam 优化算法，最大学习率为 2.5e-4。学习率在第一个 2000 更新中线性增加，然后使用 cosine schedule 减少到0。在 64 个随机选择、512 个连续序列 tokens 的 minibatch 上训练 100 个 epochs。此外，文章还使用了 layernorm，L2 正则的修改版本。使用 GELU 激活函数，使用学习的位置 embeddings，而不是原始 Transformer 论文中使用的 sinusoidal 版本。

3. Fine-tuning 细节

使用预训练阶段的超参数，分类器的 dropout rate 为 0.1。对大部分任务，学习率为 6.25e-5，batchsize 为 32。文章的实验结果表明，fine-tuning 过程很快，大多数情况下，3 个 epochs 就够了。使用 linear learning rate decay schedule，warmup 为 0.2%，$\lambda$ 设为 0.5。